In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15442, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 15442 (delta 1), reused 6 (delta 0), pack-reused 15424
Receiving objects: 100% (15442/15442), 14.05 MiB | 24.97 MiB/s, done.
Resolving deltas: 100% (10367/10367), done.


In [3]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [4]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [5]:
# make darknet (build)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [6]:
# get yolov3 pretrained coco dataset weights
!wget https://pjreddie.com/media/files/yolov3.weights

--2022-07-03 09:23:55--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  45.2MB/s    in 5.6s    

2022-07-03 09:24:01 (42.4 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [7]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [8]:
!ls '/content/drive/MyDrive/yolov3'

backup	Dataset.zip  generate_train.py	obj.data  obj.names  yolov3_custom.cfg


In [9]:
# copy the .zip file into the root directory of cloud VM
!cp /content/drive/MyDrive/yolov3/Dataset.zip ../

In [10]:
# unzip the zip file and its contents should now be in /darknet/data/Dataset
!unzip ../Dataset.zip -d data/

Archive:  ../Dataset.zip
  inflating: data/Dataset/b (1).jpg  
  inflating: data/Dataset/b (1).txt  
  inflating: data/Dataset/b (10).jpg  
  inflating: data/Dataset/b (10).txt  
  inflating: data/Dataset/b (100).jpg  
  inflating: data/Dataset/b (100).txt  
  inflating: data/Dataset/b (101).jpg  
  inflating: data/Dataset/b (101).txt  
  inflating: data/Dataset/b (102).jpg  
  inflating: data/Dataset/b (102).txt  
  inflating: data/Dataset/b (103).jpg  
  inflating: data/Dataset/b (103).txt  
  inflating: data/Dataset/b (104).jpg  
  inflating: data/Dataset/b (104).txt  
  inflating: data/Dataset/b (105).jpg  
  inflating: data/Dataset/b (105).txt  
  inflating: data/Dataset/b (106).jpg  
  inflating: data/Dataset/b (106).txt  
  inflating: data/Dataset/b (107).jpg  
  inflating: data/Dataset/b (107).txt  
  inflating: data/Dataset/b (108).jpg  
  inflating: data/Dataset/b (108).txt  
  inflating: data/Dataset/b (109).jpg  
  inflating: data/Dataset/b (109).txt  
  inflating: data/Dat

In [ ]:
# download cfg to google drive and change its name
!cp cfg/yolov3.cfg /content/drive/MyDrive/yolov3/yolov3_custom2.cfg

In [ ]:
# to download to local machine (change its name to yolov3_custom.cfg once you download)
download('cfg/yolov3.cfg')

In [11]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /content/drive/MyDrive/yolov3/yolov3_custom.cfg ./cfg

In [12]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /content/drive/MyDrive/yolov3/obj.names ./data
!cp /content/drive/MyDrive/yolov3/obj.data  ./data

In [13]:
# upload the generate_train.py script to cloud VM from Google Drive
!cp /content/drive/MyDrive/yolov3/generate_train.py ./

In [14]:
!python generate_train.py

In [15]:
# verify train.txt can be seen in our darknet/data folder
!ls data/

9k.tree     dog.jpg	 horses.jpg		   obj.data	     scream.jpg
coco9k.map  eagle.jpg	 imagenet.labels.list	   obj.names	     train.txt
coco.names  giraffe.jpg  imagenet.shortnames.list  openimages.names  voc.names
Dataset     goal.txt	 labels			   person.jpg


In [16]:
# upload pretrained convolutional layer weights
!wget http://pjreddie.com/media/files/darknet53.conv.74

URL transformed to HTTPS due to an HSTS policy
--2022-07-03 09:26:26--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  44.6MB/s    in 3.8s    

2022-07-03 09:26:31 (40.7 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
# train our custom detector
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000002, iou_loss = 0.000000, total_loss = 0.000002 
 total_bbox = 199734, rewritten_bbox = 0.019025 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.695920), count: 10, class_loss = 1.634094, iou_loss = 0.987948, total_loss = 2.622043 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000009, iou_loss = 0.000000, total_loss = 0.000009 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000002, iou_loss = 0.000000, total_loss = 0.000002 
 total_bbox = 199744, rewritten_bbox = 0.019024 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.729326), count: 11, class_loss = 2.411332, iou_loss = 1.347060, total_loss = 3.758392 
v3 

In [ ]:
# kick off training from where it last saved
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg /content/drive/MyDrive/yolov3/backup/yolov3_custom_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 149469, rewritten_bbox = 0.022078 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.811297), count: 6, class_loss = 0.780025, iou_loss = 0.229534, total_loss = 1.009559 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000561, iou_loss = 0.000000, total_loss = 0.000561 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 149475, rewritten_bbox = 0.022077 % 
]2;2279/8000: loss=0.6 hours left=13.2
 2279: 0.630965, 0.657582 avg loss, 0.001000 rate, 6.534820 seconds, 145856 images, 13.210167 hours left
Loaded: 1.195762 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.881739), count: 6, class_loss = 0.553157,